<a href="https://colab.research.google.com/github/deepshikhaa07/Facial-Emotion-Recognition-Using-Deep-Learning/blob/main/DLP_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import dataset



In [ ]:
import zipfile
import os

with zipfile.ZipFile("fer2013.zip", 'r') as zip_ref:
    zip_ref.extractall("fer2013")

# Set directory paths
train_dir = "fer2013/train"
test_dir = "fer2013/test"

## Data processing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 48
batch_size = 64

# Grayscale generators for Basic CNN
gray_train_gen = ImageDataGenerator(rescale=1./255, rotation_range=15, zoom_range=0.2, horizontal_flip=True)
gray_test_gen = ImageDataGenerator(rescale=1./255)

gray_train_data = gray_train_gen.flow_from_directory(
    train_dir, target_size=(img_size, img_size), color_mode='grayscale',
    batch_size=batch_size, class_mode='categorical', shuffle=True)

gray_test_data = gray_test_gen.flow_from_directory(
    test_dir, target_size=(img_size, img_size), color_mode='grayscale',
    batch_size=batch_size, class_mode='categorical', shuffle=False)

# RGB generators for pretrained models
rgb_train_data = gray_train_gen.flow_from_directory(
    train_dir, target_size=(img_size, img_size), color_mode='rgb',
    batch_size=batch_size, class_mode='categorical', shuffle=True)

rgb_test_data = gray_test_gen.flow_from_directory(
    test_dir, target_size=(img_size, img_size), color_mode='rgb',
    batch_size=batch_size, class_mode='categorical', shuffle=False)

num_classes = len(gray_train_data.class_indices)


## Basic CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
cnn_history = cnn_model.fit(gray_train_data, epochs=10, validation_data=gray_test_data)

#  Save model
cnn_model.save("basic_cnn_model.h5")


## EfficientNetB0 Model

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

base = EfficientNetB0(include_top=False, input_shape=(48, 48, 3), weights='imagenet')
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
out = Dense(num_classes, activation='softmax')(x)
eff_model = Model(inputs=base.input, outputs=out)

eff_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
eff_history = eff_model.fit(rgb_train_data, epochs=10, validation_data=rgb_test_data)

# Save model
eff_model.save("efficientnet_model.h5")


## ResNet50 Model

In [ ]:
from tensorflow.keras.applications import ResNet50

base = ResNet50(include_top=False, input_shape=(48, 48, 3), weights='imagenet')
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
out = Dense(num_classes, activation='softmax')(x)
resnet_model = Model(inputs=base.input, outputs=out)

resnet_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
resnet_history = resnet_model.fit(rgb_train_data, epochs=10, validation_data=rgb_test_data)

# Save model
resnet_model.save("resnet50_model.h5")


## MobileNetV2

In [ ]:
from tensorflow.keras.applications import MobileNetV2

base = MobileNetV2(include_top=False, input_shape=(48, 48, 3), weights='imagenet')
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
out = Dense(num_classes, activation='softmax')(x)
mobile_model = Model(inputs=base.input, outputs=out)

mobile_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
mobile_history = mobile_model.fit(rgb_train_data, epochs=10, validation_data=rgb_test_data)

# Save model
mobile_model.save("mobilenetv2_model.h5")


## Evaluation of Models

In [ ]:
# Evaluate accuracy
cnn_loss, cnn_acc = cnn_model.evaluate(gray_test_data)
eff_loss, eff_acc = eff_model.evaluate(rgb_test_data)
resnet_loss, resnet_acc = resnet_model.evaluate(rgb_test_data)
mobile_loss, mobile_acc = mobile_model.evaluate(rgb_test_data)

print(f"CNN Accuracy: {cnn_acc:.2f}")
print(f"EfficientNet Accuracy: {eff_acc:.2f}")
print(f"ResNet50 Accuracy: {resnet_acc:.2f}")
print(f"MobileNetV2 Accuracy: {mobile_acc:.2f}")


## Visualisation

In [ ]:
# Evaluate confusion matrix

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np


models = [cnn_model, eff_model, resnet_model, mobile_model]
test_data = [gray_test_data, rgb_test_data, rgb_test_data, rgb_test_data]
model_names = ['CNN', 'EfficientNetB0', 'ResNet50', 'MobileNetV2']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, (model, data, name) in enumerate(zip(models, test_data, model_names)):
    y_pred = model.predict(data)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = data.classes
    cm = confusion_matrix(y_true, y_pred_classes)

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'Confusion Matrix - {name}')
    axes[i].set_xlabel('Predicted Label')
    axes[i].set_ylabel('True Label')

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate classification report

from sklearn.metrics import classification_report

for model, data, name in zip(models, test_data, model_names):
    y_pred = model.predict(data)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = data.classes
    report = classification_report(y_true, y_pred_classes, target_names=data.class_indices.keys())
    print(f"Classification Report - {name}:\n{report}\n")